# Attribute-based Text Generation from Amazon Product Reviews

## Library Imports

In [ ]:
from numpy import zeros, concatenate, asarray
from IPython.display import display, HTML

## Notification JavaScript snippets

In [ ]:
def browser_alert(message):
    display(HTML('<script type="text/javascript">alert("' + message + '");</script>'))
    
def browser_notify(message):
    display(HTML('<script type="text/javascript">var notification=new Notification("' + \
                 'Jupyter Notification",{icon:"http://blog.jupyter.org/content/' + \
                 'images/2015/02/jupyter-sq-text.png",body:"' + message + \
                 '"});</script>'))

In [ ]:
browser_notify("test")

## Reading Data

In [ ]:
dataset_path = "/home/v2john/attr-reviews-dataset/"

In [ ]:
dev_set_path = dataset_path + "dev.txt"
train_set_path = dataset_path + "train.txt"
test_set_path = dataset_path + "test.txt"
vocab_path = dataset_path + "vocab.txt"

In [ ]:
def build_vocab_index(vocab_file_path):
    
    vocab_index = dict()
    current_index = 0
    
    with open(vocab_file_path) as vocab_file:
        for line in vocab_file:
            word = line.split()[0]
            vocab_index[word] = current_index
            current_index += 1
            
    return vocab_index


def build_user_and_product_indices(review_file_path):
    
    user_index = dict()
    product_index = dict()
    
    current_user_index = 0
    current_product_index = 0
    
    with open(review_file_path) as review_file:
        for line in review_file:
            split_line = line.split()
            user_id = split_line[0]
            product_id = split_line[1]
            
            if user_id not in user_index:
                user_index[user_id] = current_user_index
                current_user_index += 1
                
            if product_id not in product_index:
                product_index[product_id] = current_product_index
                current_product_index += 1
                
    return user_index, product_index


def build_rating_index():
    
    rating_index = dict()
    
    for i in range(0, 5):
        rating_index[(i + 1) * 1.0] = i
        
    return rating_index

In [ ]:
vocab_index = build_vocab_index(vocab_path)
print(str(len(vocab_index)) + " terms in the vocabulary")

In [ ]:
user_index, product_index = build_user_and_product_indices(dev_set_path)
print(str(len(user_index)) + " distinct users")
print(str(len(product_index)) + " distinct products")

In [ ]:
rating_index = build_rating_index()
print(str(len(rating_index)) + " distinct ratings")

In [ ]:
browser_notify("Indices built")

## Convert build attribute and text-embedding versions of the training data

In [ ]:
from nltk import word_tokenize

In [ ]:
def get_one_hot_vector_embedding(term_index, term):
    return asarray([term_index[term]])

In [ ]:
def get_review_sequence(review, vocab_index):
    
    review_sequence = list()
    
    tokens = word_tokenize(review)
    for token in tokens:
        try:
            word_index = get_one_hot_vector_embedding(vocab_index, str(token))
            review_sequence.append(word_index)
        except KeyError as ke:
            pass  # skip missing keys
    
    return asarray(review_sequence)

def get_attribute_vectors_and_text_embedding_sequences(review_file_path):

    x_train_att = list()
    x_train_rev = list()
    i = 0
    
    with open(review_file_path) as review_file:
        for line in review_file:
            split_line = line.split("\t")
            user_id = split_line[0]
            product_id = split_line[1]
            rating = float(split_line[2])
            review_text = split_line[3]
            
            attribute_vector = concatenate((get_one_hot_vector_embedding(user_index, user_id), 
                                            get_one_hot_vector_embedding(product_index, product_id),
                                            get_one_hot_vector_embedding(rating_index, rating)))
            
            x_train_att.append(attribute_vector)
            x_train_rev.append(get_review_sequence(review_text, vocab_index))
            
#             i += 1
#             if i == 10:
#                 break
            
    return asarray(x_train_att), asarray(x_train_rev)

In [ ]:
x_train_att, x_train_rev = get_attribute_vectors_and_text_embedding_sequences(dev_set_path)

In [ ]:
x_train_att.shape, x_train_att[0].shape

In [ ]:
x_train_rev.shape, x_train_rev[0].shape

### Save training attributes

In [ ]:
import pickle

In [ ]:
flatfiles_path = "/home/v2john/text-gen-flatfiles/"

In [ ]:
# Save vectors

with open(flatfiles_path + 'x_train_att.pkl', 'wb') as x_train_att_file:
    pickle.dump(x_train_att, x_train_att_file)

with open(flatfiles_path + 'x_train_rev.pkl', 'wb') as x_train_rev_file:
    pickle.dump(x_train_rev, x_train_rev_file)

In [ ]:
# # Restore vectors

# with open(flatfiles_path + 'x_train_att.pkl', 'rb') as x_train_att_file:
#     x_train_att = pickle.load(x_train_att_file)
    
# with open(flatfiles_path + 'x_train_rev.pkl', 'rb') as x_train_rev_file:
#     x_train_rev = pickle.load(x_train_rev_file)

## Define Keras Model

In [ ]:
from keras.layers import Input, Dense, RepeatVector, LSTM, Conv1D
from keras.models import Model

In [ ]:
inputs = Input(shape=x_train_att[0].shape)
print(x_train_att[0])
x = Dense(64, activation='relu')(inputs)
print(x)
x = Dense(1, activation='relu')(inputs)
print(x)
predictions = RepeatVector(10)(x)
print(predictions)
# predictions = Conv1D(64, activation='relu', kernel_size=3)(x)
# print(predictions)

model = Model(inputs, predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train_att, x_train_rev)